In [4]:
import pandas as pd
import numpy as np
from obfuscator import comment_remover, variable_renamer
from csv_stuff import create_labled_table_routine, add_valid_code_columns, remove_given_code
import sys
# sys.path.append('/home/erik/TU/ni/Source-Code-Plagiarism-Detection/Source codes/')
sys.path.append('../../../Source-Code-Plagiarism-Detection/Source codes/')
from remove_given_code_from_student_solution import remove_given_code_from_df, remove_given_code_from_df_WS2021


PATH_TEMPLATE = "/home/erik/TU/ni/plagiate_labeltool/data/code_templates/"


def create_plagiate_table(df, semester, ha, task, prog_language, number_labled_pairs, path):
    df_labled = pd.DataFrame(columns=['semester', 'ha', 'task', 'prog_lang',
                                      'surname1', 'lastname1', 'code1', 'code2', 'label'])

    df = add_valid_code_columns(df, semester, ha, [task], prog_language)
    number_of_valid_codes = len(df.loc[df[f'{task} empty'] == 0])
    print("number_of_valid_codes: " + str(number_of_valid_codes))
    plagiate_per_solution = int(np.floor(number_labled_pairs/number_of_valid_codes))
    print("plagiate_per_solution: " + str(plagiate_per_solution))
    for ln, sn, code in df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values:
        for i in range(plagiate_per_solution):
            plagiate_code = comment_remover(code)
            plagiate_code = variable_renamer(plagiate_code)
            df_labled.loc[len(df_labled)] = [
                semester, ha, task, prog_language, sn, ln, code, plagiate_code, 1]
    df_labled.to_csv(path)
    return df_labled


def create_plagiate_table_routine(semester, ha, task, prog_language):
    csv_path = f'../../data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    csv_path1 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_removed_preload.csv'
    # csv_path1 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_labled.csv'
    csv_path2 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    df = pd.read_csv(csv_path, delimiter=',')
    number_labled_pairs = len(pd.read_csv(csv_path1, delimiter=','))
    df_labled = create_plagiate_table(
        df, semester, ha, task, prog_language, number_labled_pairs, csv_path2)
    return df_labled


def create_plagiate_table_from_df(semester, ha, prog_language):
    labled_csv = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_removed_preload.csv'
    save_path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    df = pd.read_csv(labled_csv, delimiter=',')
    number_labled_pairs = len(df)
    df = df.drop_duplicates(subset=['surname1'])
    number_of_valid_codes = len(df)
    df = remove_given_code_from_df(df)
    print("number_of_valid_codes: " + str(number_of_valid_codes))
    print("number_labled_pairs: " + str(number_labled_pairs))
    plagiate_per_solution = int(np.floor(number_labled_pairs/number_of_valid_codes))
    print("plagiate_per_solution: " + str(plagiate_per_solution))
    # drop column hand_labled
    df = df.drop(columns=['hand_labled', 'Unnamed: 0', 'surname2', 'lastname2'])
    # duplicate each row of df1 plagiate_per_solution times and create new column code2 and use the field code1 as input value for the functions comment_remover and variable_renamer
    df = df.reindex(df.index.repeat(plagiate_per_solution))
    df['code2'] = df['code1'].apply(comment_remover)
    df['code2'] = df['code2'].apply(variable_renamer)
    df['label'] = 1
    df.to_csv(save_path)
    return df, plagiate_per_solution


def open_csvs(semester, ha, task, prog_language):
    original_csv_path = f'../../data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    labled_csv_path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_removed_preload.csv'
    plagiate_csv_path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    df_original = pd.read_csv(original_csv_path, delimiter=',')
    df_labled = pd.read_csv(labled_csv_path, delimiter=',')
    return df_original, df_labled, plagiate_csv_path


In [ ]:
def split_data_from_labled(semester, ha, task, prog_language):
    df_labled = pd.read_csv(f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_labled.csv', delimiter=',')
    if 'WS' in semester:
        remove_given_code_from_df_WS2021(df_labled)
    else:
        remove_given_code_from_df(df_labled)
    create_plagiate_table_from_df(semester, ha, prog_language)


In [11]:
# df_original, df_labled, plagiate_csv_path = open_csvs(
#     'WS2021', '9', 'Antwort 9', 'C')

In [12]:
semester='WS2021'
ha='9'
task='Antwort 9'
prog_language='C'
# path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
df_plagiate, plagiate_per_solution = create_plagiate_table_from_df(semester=semester, ha=ha, prog_language=prog_language)


number_of_valid_codes: 20
number_labled_pairs: 210
plagiate_per_solution: 10


In [ ]:
#TODO create_plagiate_table_from_df ziel:
#   alle vorgaben aus den codes vom df_labled entfernen und speichern
#   daraus dann die plagiate erstellen und speichern

# Plagiate maker with pycparser

In [18]:
from pycparser import c_generator, c_parser
import pycparser

# Parse C code
c_code = '''
int main() {
    int a = 1;
    int b = 2;
    int c = a + b;
    return c;
}
'''
# store the code in a file
with open('test.c', 'w') as f:
    f.write(c_code)
    f.close()
ast=pycparser.parse_file('test.c')
# generate Plagiat from AST
import random
import string

def generate_plagiat(ast, n):
    '''Generate n new plagiat based on AST'''
    plagiat = []
    variables = []
    for i in range(n):
        # replace variable names
        for node in ast.ext:
            if isinstance(node, pycparser.c_ast.FuncDef):
                for stmt in node.body.block_items:
                    if isinstance(stmt, pycparser.c_ast.Decl):
                        variables.append(stmt.type.declname)
                        stmt.type.declname = ''.join(random.choices(string.ascii_lowercase, k=5))
                    # print(stmt)


        # replace variable names in expressions
        for node in ast.ext:
            if isinstance(node, pycparser.c_ast.FuncDef):
                for stmt in node.body.block_items:
                    if isinstance(stmt, pycparser.c_ast.BinaryOp):
                        for var in variables:
                            if var in stmt.left.name:
                                stmt.left.name = ''.join(random.choices(string.ascii_lowercase, k=5))
                            if var in stmt.right.name:
                                stmt.right.name = ''.join(random.choices(string.ascii_lowercase, k=5))

        # change function names
        for node in ast.ext:
            if isinstance(node, pycparser.c_ast.FuncDef):
                node.decl.name = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
                node.decl.type.type.declname = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
                
        # ast to string

        code= c_generator.CGenerator().visit(ast)
        plagiat.append(code)
    return plagiat

        
plagiat = generate_plagiat(ast, 5)
# print(ast)
# for p in ast:
#     print(p)
print(c_generator.CGenerator().visit(ast))

int PKW24()
{
  int jkjlk = 1;
  int cqhjd = 2;
  int ryguu = a + b;
  return c;
}


